# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298300678626                   -0.85    5.0         
  2   -8.300191673884       -2.72       -1.25    1.0    110ms
  3   -8.300432764514       -3.62       -1.89    2.9    143ms
  4   -8.300462422358       -4.53       -2.79    2.0    131ms
  5   -8.300464425564       -5.70       -3.11    3.0    275ms
  6   -8.300464535229       -6.96       -3.28    1.1   97.3ms
  7   -8.300464586569       -7.29       -3.44    8.6    199ms
  8   -8.300464617688       -7.51       -3.61    1.6    117ms
  9   -8.300464635143       -7.76       -3.78    1.8    125ms
 10   -8.300464637979       -8.55       -3.85    8.2    245ms
 11   -8.300464642642       -8.33       -4.08    2.4    182ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.64166598077                   -0.70    6.1         
  2   -16.67811282045       -1.44       -1.14    1.4    245ms
  3   -16.67920070791       -2.96       -1.88    3.9    354ms
  4   -16.67927991748       -4.10       -2.75    2.2    290ms
  5   -16.67928600432       -5.22       -3.11    4.5    376ms
  6   -16.67928620265       -6.70       -3.52    3.0    327ms
  7   -16.67928621933       -7.78       -4.00    2.4    290ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32819951212                   -0.56    7.0         
  2   -33.33472961100       -2.19       -1.00    1.5    1.00s
  3   -33.33596713375       -2.91       -1.72    5.1    1.30s
  4   -33.33612826365       -3.79       -2.62    5.0    1.21s
  5   -33.33679808351       -3.17       -2.24    7.4    1.70s
  6   -33.33682416205       -4.58       -2.29    2.0    965ms
  7   -33.33694316495       -3.92       -3.49    1.6    1.06s
  8   -33.33694338297       -6.66       -3.50    5.1    1.50s
  9   -33.33694356006       -6.75       -3.63    1.4    908ms
 10   -33.33694377272       -6.67       -4.34    1.6    996ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298225675620                   -0.85    4.8         
  2   -8.300235242241       -2.70       -1.58    1.2   85.7ms
  3   -8.300433542312       -3.70       -2.60    4.1    131ms
  4   -8.300398619955   +   -4.46       -2.34    3.0    136ms
  5   -8.300464254237       -4.18       -3.45    1.0   82.6ms
  6   -8.300464553217       -6.52       -3.78    3.0    132ms
  7   -8.300464639860       -7.06       -4.31    1.5    112ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32498397926                   -0.56    5.6         
  2   -33.28887213166   +   -1.44       -1.25    1.0    793ms
  3   +6.799402638515   +    1.60       -0.26    7.0    2.03s
  4   -33.32851919882        1.60       -1.77    6.4    1.80s
  5   -33.23005853922   +   -1.01       -1.33    4.8    1.57s
  6   -32.62228163814   +   -0.22       -1.14    4.5    1.40s
  7   -33.26586845448       -0.19       -1.60    5.1    1.41s
  8   -33.33457800417       -1.16       -2.24    3.4    1.13s
  9   -33.33522987859       -3.19       -2.37    3.5    1.16s
 10   -33.33673219758       -2.82       -2.54    2.8    1.13s
 11   -33.33692203823       -3.72       -3.00    1.8    944ms
 12   -33.33693840506       -4.79      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.